# Исследование базы данных о книгах
____

### В данном исследовании анализируется информация о книгах  для формирования цен.
____

### Исследование разделено на несколько частей.

#### [Часть 1. Подготовка к исследованию:](#1)

* [1. Импорт библиотек.](#1.1)
* [2. Подготовка к подключению к БД.](#1.2)
* [3. Получение общей информации о данных.](#1.3)

#### [Часть 2. Анализ данных:](#2)

#### [Часть 3. Общий вывод](#3)

### Часть 1. Подготовка к исследованию и изучение общей информации о данных. <a id='1'></a>

#### 1.1 Импорт библиотек <a id='1.1'></a>

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt

#### 1.2 Подготовка к подключению к БД. <a id='1.2'></a>

In [2]:

def get_data(q):
    return pd.io.sql.read_sql(q, con = engine)

#### 1.3 Получение и изучение общей информации о данных. <a id='1.3'></a>

Создадим функцию для вывода первых 5 строк таблицы из БД.

In [3]:
def print_n_lines(lines, data):
    query = '''
            SELECT 
                *
            FROM 
                {}
            LIMIT 
                {}
            '''.format(data, lines)
    
    display(get_data(query))

Выведем по 5 строк для каждой таблицы, необходимой для исследования.

In [4]:
datasets = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for data in datasets:
    print_n_lines(5, data)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Часть 2. Анализ данных. <a id='2'></a>

Посчитаем сколько книг вышло с 1 января 2000 года.

In [5]:
cnt_books = '''
            SELECT
                COUNT(DISTINCT book_id) AS cnt_books
            FROM 
                books
            WHERE
                publication_date :: date > '2000-01-01'
            '''

display(get_data(cnt_books))

,cnt_books
0,819


Для каждой книги посчитаем количество отзывов и среднюю оценку.

In [6]:
review_and_rating = '''
                    SELECT
                        books.book_id,
                        books.title,
                        subdata.cnt_review,
                        subdata.av_rating
                    FROM books
                    INNER JOIN
                        (
                         SELECT
                             books.book_id,
                             AVG(ratings.rating) AS av_rating,
                             COUNT(DISTINCT reviews.review_id) AS cnt_review
                         FROM
                             books
                         LEFT JOIN ratings ON ratings.book_id = books.book_id
                         LEFT JOIN reviews ON reviews.book_id = books.book_id
                         GROUP BY
                             books.book_id
                        ) AS subdata ON subdata.book_id = books.book_id
                    '''
display(get_data(review_and_rating).head())

,book_id,title,cnt_review,av_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000


Выведем топ 5 издательств по количеству книг. Будем учитывать только книги, с количеством страниц больше 50. Это сделано для исключения брошюр из анализа.

In [7]:
top_publishers = '''
                 SELECT
                     publishers.publisher_id,
                     publishers.publisher,
                     COUNT(DISTINCT books.book_id) AS cnt_books
                 FROM
                     books
                 LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
                 WHERE books.num_pages > 50
                 GROUP BY
                     publishers.publisher_id,
                     publishers.publisher
                 ORDER BY
                     cnt_books DESC
                 '''
display(get_data(top_publishers).head())

,publisher_id,publisher,cnt_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


Больше всего книг в нашей базе от издательства `Penguin Books`. Причем с довольно большим отрывом: 11 книг.

Выведем топ 5 авторов с самой высокой средней оценкой книг. Книги с маленьким количеством оценок (меньше 50) учитывать не будем.

In [8]:
top_author = '''
             WITH top_books AS (
                 SELECT
                     authors.author_id,
                     authors.author,
                     books.book_id
                 FROM
                     ratings             
                 LEFT JOIN books ON books.book_id = ratings.book_id
                 LEFT JOIN authors ON authors.author_id = books.author_id
                 GROUP BY
                     authors.author_id,
                     authors.author,
                     books.book_id
                 HAVING
                     COUNT(ratings.rating_id) >= 50
             ) 
             SELECT
                 top_books.author_id,
                 top_books.author,
                 AVG(ratings.rating) AS avg_rating
             FROM
                 top_books             
             LEFT JOIN ratings ON top_books.book_id = ratings.book_id 
             GROUP BY
                 top_books.author_id,
                 top_books.author
             ORDER BY
                 avg_rating DESC
             '''

display(get_data(top_author).head())

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


Лучший писатель, по мнению пользователей, Роулинг! Вместе с иллюстрациями Грандпри.

Посчитаем среднее количество обзоров от активных пользователей.

In [9]:
avg_reviews = '''
              WITH
                  active_users AS (
                      SELECT
                          username
                      FROM
                          ratings
                      GROUP BY
                          username
                      HAVING COUNT(rating_id) > 50
                  ),
                  user_reviews AS (
                      SELECT
                          username,
                          COUNT(review_id) AS cnt_reviews
                      FROM
                          reviews
                      WHERE username IN (SELECT username FROM active_users)
                      GROUP BY
                          username
                  )
              SELECT
                  AVG(cnt_reviews) AS avg_reviews
              FROM
                  user_reviews
              '''

display(get_data(avg_reviews).head())

,avg_reviews
0,24.333333


В среднем активные пользователи пишут чуть больше 24 обзоров.

### 3. Общий вывод <a id='3'></a>

- Больше всего книг в нашей базе от издательства `Penguin Books`. Причем с довольно большим отрывом: 11 книг.
- Лучший писатель, по мнению пользователей, Роулинг! Вместе с иллюстрациями Грандпри.
- В среднем активные пользователи пишут чуть больше 24 обзоров.